In [34]:
import numpy as np
from tqdm import tqdm

In [35]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.models import load_model


## Prepare driving data

In [36]:
import glob
speed_file_name_ls = glob.glob('./TrainingData/All_in_one/truth_*.npy')
print('total number of unbiased data points:',len(speed_file_name_ls))

total number of unbiased data points: 2000


In [37]:
speed_file_name_ls[0]

'./TrainingData/All_in_one/truth_36070V15.npy'

In [38]:
# initial point
speed_f = speed_file_name_ls[0]
all_y5 = np.load(speed_f)
data_index = speed_f.split('truth_')[1].split('V')[0]
all_images = np.load('./TrainingData/All_in_one/image_'+data_index+'.npy')
all_rnn = np.load('./TrainingData/All_in_one/rnn_'+data_index+'.npy')

In [39]:
for j in tqdm(range(1,1000)):
    speef_f = speed_file_name_ls[j]
    truth_speed = np.load(speed_f)
    data_index = speed_f.split('truth_')[1].split('V')[0]
    image = np.load('./TrainingData/All_in_one/image_'+data_index+'.npy')
    rnn = np.load('./TrainingData/All_in_one/rnn_'+data_index+'.npy')
    all_images = np.vstack((all_images, image))
    all_rnn = np.vstack((all_rnn, rnn))
    all_y5 = np.vstack((all_y5, truth_speed))

100%|██████████| 999/999 [03:20<00:00,  4.98it/s]


In [40]:
num_data = all_images.shape[0]


In [41]:
all_desire = np.zeros((num_data,8))

all_y0 = np.zeros((num_data,385))
all_y1 = np.zeros((num_data,386))
all_y2 = np.zeros((num_data,386))
all_y3 = np.zeros((num_data,58))
all_y4 = np.zeros((num_data,200))
all_y6 = np.zeros((num_data,200))
all_y7 = np.zeros((num_data,4))
all_y8 = np.zeros((num_data,32))
all_y9 = np.zeros((num_data,12))
all_y10 = np.zeros((num_data,512))

In [42]:
x_all = [all_images,all_desire,all_rnn]
y_all = [all_y0,all_y1,all_y2,all_y3,all_y4,all_y5,all_y6,all_y7,all_y8,all_y9,all_y10]

### Training

In [43]:
model = load_model('pretrain_model.keras')

In [44]:
for i in range(len(model.layers)):
    if model.layers[i].name =='dense_1_long_v' or model.layers[i].name =='dense_2_long_v' or \
    model.layers[i].name =='dense_3_long_v':
        model.layers[i].trainable = True
    else:
        model.layers[i].trainable = False

for i in range(len(model.layers)):
    if model.layers[i].trainable == True:
        print('layer name {}, trainable: {}'.format(model.layers[i].name, model.layers[i].trainable))

layer name dense_1_long_v, trainable: True
layer name dense_2_long_v, trainable: True
layer name dense_3_long_v, trainable: True


In [45]:
# define a customized loss function to compare the predicted v and real v 
def speed_loss(y_true, y_pred):
    print('see y_true: ',y_true)
    batch_size = y_pred.shape[0]
    loss=0
    for i in range(batch_size):
        pred_v = y_pred[i,:]
        true_v = y_true[i,:]
        for j in range(10):
            loss+=abs(pred_v[i*10]-true_v[i*10])
    return loss

def zero_loss(y_true,y_pred):
    return 0.0

model.compile(optimizer=Adam(lr=1e-3),loss={'path': zero_loss, 'left_lane': zero_loss, \
                                            'right_lane': zero_loss, 'lead': zero_loss, \
                                           'long_x':zero_loss, 'long_v':speed_loss, \
                                           'long_a':zero_loss, 'meta':zero_loss,\
                                           'snpe_pleaser2':zero_loss, 'pose':zero_loss,\
                                           'add_3':zero_loss})

In [46]:
log_dir = 'logs/unbiased/'
logging = TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)

epi_hist = model.fit(x_all,y_all, batch_size=4, epochs=100, validation_split=0.2,callbacks=[logging, checkpoint])
np.save('epi_loss_his',epi_hist.history['loss'])
np.save('epi_val_loss_his',epi_hist.history['val_loss'])

Epoch 1/100
see y_true:  Tensor("IteratorGetNext:8", shape=(4, 200), dtype=float32)
see y_true:  Tensor("IteratorGetNext:8", shape=(4, 200), dtype=float32)
200/200 [==============================] - 18s 88ms/step - loss: 0.9948 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.9948 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 - snpe_pleaser2_loss: 0.0000e+00 - pose_loss: 0.0000e+00 - add_3_loss: 0.0000e+00 - val_loss: 0.4968 - val_path_loss: 0.0000e+00 - val_left_lane_loss: 0.0000e+00 - val_right_lane_loss: 0.0000e+00 - val_lead_loss: 0.0000e+00 - val_long_x_loss: 0.0000e+00 - val_long_v_loss: 0.4968 - val_long_a_loss: 0.0000e+00 - val_meta_loss: 0.0000e+00 - val_snpe_pleaser2_loss: 0.0000e+00 - val_pose_loss: 0.0000e+00 - val_add_3_loss: 0.0000e+00
Epoch 2/100
200/200 [==============================] - 16s 82ms/step - loss: 1.0294 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+0

Epoch 11/100
200/200 [==============================] - 16s 81ms/step - loss: 0.3592 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.3592 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 - snpe_pleaser2_loss: 0.0000e+00 - pose_loss: 0.0000e+00 - add_3_loss: 0.0000e+00 - val_loss: 0.3821 - val_path_loss: 0.0000e+00 - val_left_lane_loss: 0.0000e+00 - val_right_lane_loss: 0.0000e+00 - val_lead_loss: 0.0000e+00 - val_long_x_loss: 0.0000e+00 - val_long_v_loss: 0.3821 - val_long_a_loss: 0.0000e+00 - val_meta_loss: 0.0000e+00 - val_snpe_pleaser2_loss: 0.0000e+00 - val_pose_loss: 0.0000e+00 - val_add_3_loss: 0.0000e+00
Epoch 12/100
200/200 [==============================] - 16s 82ms/step - loss: 0.3427 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.3427 - long_a_loss: 0.0000e+00 - meta_loss:

200/200 [==============================] - 18s 91ms/step - loss: 0.3589 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.3589 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 - snpe_pleaser2_loss: 0.0000e+00 - pose_loss: 0.0000e+00 - add_3_loss: 0.0000e+00 - val_loss: 0.1613 - val_path_loss: 0.0000e+00 - val_left_lane_loss: 0.0000e+00 - val_right_lane_loss: 0.0000e+00 - val_lead_loss: 0.0000e+00 - val_long_x_loss: 0.0000e+00 - val_long_v_loss: 0.1613 - val_long_a_loss: 0.0000e+00 - val_meta_loss: 0.0000e+00 - val_snpe_pleaser2_loss: 0.0000e+00 - val_pose_loss: 0.0000e+00 - val_add_3_loss: 0.0000e+00
Epoch 23/100
200/200 [==============================] - 17s 84ms/step - loss: 0.3933 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.3933 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 -

200/200 [==============================] - 17s 87ms/step - loss: 0.2483 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.2483 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 - snpe_pleaser2_loss: 0.0000e+00 - pose_loss: 0.0000e+00 - add_3_loss: 0.0000e+00 - val_loss: 0.4102 - val_path_loss: 0.0000e+00 - val_left_lane_loss: 0.0000e+00 - val_right_lane_loss: 0.0000e+00 - val_lead_loss: 0.0000e+00 - val_long_x_loss: 0.0000e+00 - val_long_v_loss: 0.4102 - val_long_a_loss: 0.0000e+00 - val_meta_loss: 0.0000e+00 - val_snpe_pleaser2_loss: 0.0000e+00 - val_pose_loss: 0.0000e+00 - val_add_3_loss: 0.0000e+00
Epoch 34/100
200/200 [==============================] - 21s 106ms/step - loss: 0.3040 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.3040 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 

200/200 [==============================] - 16s 81ms/step - loss: 0.4146 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.4146 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 - snpe_pleaser2_loss: 0.0000e+00 - pose_loss: 0.0000e+00 - add_3_loss: 0.0000e+00 - val_loss: 0.1569 - val_path_loss: 0.0000e+00 - val_left_lane_loss: 0.0000e+00 - val_right_lane_loss: 0.0000e+00 - val_lead_loss: 0.0000e+00 - val_long_x_loss: 0.0000e+00 - val_long_v_loss: 0.1569 - val_long_a_loss: 0.0000e+00 - val_meta_loss: 0.0000e+00 - val_snpe_pleaser2_loss: 0.0000e+00 - val_pose_loss: 0.0000e+00 - val_add_3_loss: 0.0000e+00
Epoch 45/100
200/200 [==============================] - 16s 81ms/step - loss: 0.3311 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.3311 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 -

200/200 [==============================] - 16s 81ms/step - loss: 0.2050 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.2050 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 - snpe_pleaser2_loss: 0.0000e+00 - pose_loss: 0.0000e+00 - add_3_loss: 0.0000e+00 - val_loss: 0.0374 - val_path_loss: 0.0000e+00 - val_left_lane_loss: 0.0000e+00 - val_right_lane_loss: 0.0000e+00 - val_lead_loss: 0.0000e+00 - val_long_x_loss: 0.0000e+00 - val_long_v_loss: 0.0374 - val_long_a_loss: 0.0000e+00 - val_meta_loss: 0.0000e+00 - val_snpe_pleaser2_loss: 0.0000e+00 - val_pose_loss: 0.0000e+00 - val_add_3_loss: 0.0000e+00
Epoch 56/100
200/200 [==============================] - 16s 82ms/step - loss: 0.2029 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.2029 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 -

200/200 [==============================] - 16s 81ms/step - loss: 0.1856 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.1856 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 - snpe_pleaser2_loss: 0.0000e+00 - pose_loss: 0.0000e+00 - add_3_loss: 0.0000e+00 - val_loss: 0.1643 - val_path_loss: 0.0000e+00 - val_left_lane_loss: 0.0000e+00 - val_right_lane_loss: 0.0000e+00 - val_lead_loss: 0.0000e+00 - val_long_x_loss: 0.0000e+00 - val_long_v_loss: 0.1643 - val_long_a_loss: 0.0000e+00 - val_meta_loss: 0.0000e+00 - val_snpe_pleaser2_loss: 0.0000e+00 - val_pose_loss: 0.0000e+00 - val_add_3_loss: 0.0000e+00
Epoch 67/100
200/200 [==============================] - 18s 88ms/step - loss: 0.2360 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.2360 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 -

200/200 [==============================] - 16s 81ms/step - loss: 0.1406 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.1406 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 - snpe_pleaser2_loss: 0.0000e+00 - pose_loss: 0.0000e+00 - add_3_loss: 0.0000e+00 - val_loss: 0.0410 - val_path_loss: 0.0000e+00 - val_left_lane_loss: 0.0000e+00 - val_right_lane_loss: 0.0000e+00 - val_lead_loss: 0.0000e+00 - val_long_x_loss: 0.0000e+00 - val_long_v_loss: 0.0410 - val_long_a_loss: 0.0000e+00 - val_meta_loss: 0.0000e+00 - val_snpe_pleaser2_loss: 0.0000e+00 - val_pose_loss: 0.0000e+00 - val_add_3_loss: 0.0000e+00
Epoch 78/100
200/200 [==============================] - 16s 82ms/step - loss: 0.2102 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.2102 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 -

200/200 [==============================] - 17s 84ms/step - loss: 0.1480 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.1480 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 - snpe_pleaser2_loss: 0.0000e+00 - pose_loss: 0.0000e+00 - add_3_loss: 0.0000e+00 - val_loss: 0.0340 - val_path_loss: 0.0000e+00 - val_left_lane_loss: 0.0000e+00 - val_right_lane_loss: 0.0000e+00 - val_lead_loss: 0.0000e+00 - val_long_x_loss: 0.0000e+00 - val_long_v_loss: 0.0340 - val_long_a_loss: 0.0000e+00 - val_meta_loss: 0.0000e+00 - val_snpe_pleaser2_loss: 0.0000e+00 - val_pose_loss: 0.0000e+00 - val_add_3_loss: 0.0000e+00
Epoch 89/100
200/200 [==============================] - 16s 81ms/step - loss: 0.1423 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.1423 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 -

200/200 [==============================] - 17s 85ms/step - loss: 0.1497 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.1497 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 - snpe_pleaser2_loss: 0.0000e+00 - pose_loss: 0.0000e+00 - add_3_loss: 0.0000e+00 - val_loss: 0.0676 - val_path_loss: 0.0000e+00 - val_left_lane_loss: 0.0000e+00 - val_right_lane_loss: 0.0000e+00 - val_lead_loss: 0.0000e+00 - val_long_x_loss: 0.0000e+00 - val_long_v_loss: 0.0676 - val_long_a_loss: 0.0000e+00 - val_meta_loss: 0.0000e+00 - val_snpe_pleaser2_loss: 0.0000e+00 - val_pose_loss: 0.0000e+00 - val_add_3_loss: 0.0000e+00
Epoch 100/100
200/200 [==============================] - 16s 81ms/step - loss: 0.1715 - path_loss: 0.0000e+00 - left_lane_loss: 0.0000e+00 - right_lane_loss: 0.0000e+00 - lead_loss: 0.0000e+00 - long_x_loss: 0.0000e+00 - long_v_loss: 0.1715 - long_a_loss: 0.0000e+00 - meta_loss: 0.0000e+00 